## ВКР: обработка данных

**Автоматизация подбора персонала**

**Копчев Владислав, БПМИ197**

В результате работы данного ноутбука мы предобоработаем собранные признаки: приведем их к единому виду, разобьем некоторые признаки на более мелкие, суммаризуем раздел "О себе" с помощью нейросети mBART.

Импортируем необходимые нам библиотеки:

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# !pip install transliterate
from transliterate import translit
import re
import numpy as np

In [2]:
import nltk
# nltk.download('stopwords')
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
morph = MorphAnalyzer()

### Пост-обработка

Импорт данных:

Поскольку mBART обучается очень долго (ожидаемое время — около 150–180 часов для всех данных), для уменьшения объема работы, но с сохранением репрезентативности данных при этом, было решено взять случайную выборку размером 3000 и работать с ней. 

In [3]:
df = pd.read_csv('resumes_features.csv')
# df = pd.read_csv('resumes_features.csv').sample(3000)
# df = pd.read_csv('backup/16.11/resumes_features.csv').sample(3000)  # <- после 18.04

Поскльку `df` может импортироваться с лишним столбцом в начале или неправильным индексом, запустим следующий код:

In [4]:
df.drop(list(df)[0], axis=1, inplace=True)
df.reset_index(inplace=True)
df.drop(list(df)[0], axis=1, inplace=True)

Проверим, что с таблицей и индексом все в порядке:

In [5]:
df.sample(2)

,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,...,91,92,93,94,95,96,97,98,99,clust
1109,https://hh.ru/resume/5cf55f5300051380170039ed1...,Начинающий специалист,"Москва, не готов к переезду, готов к командиро...",Ключевые навыки,На данный момент имеются следующие теоретическ...,Высшее образование (Бакалавр)\n2019\nСанкт-Пет...,Специализации:\nДругое\nЗанятость: полная заня...,---,Санкт-Петербургский государственный технологич...,NaN,...,-0.001156,0.001366,-0.000280,0.000224,-0.000075,0.000588,0.000045,0.000758,-0.000920,0
215,https://hh.ru/resume/45fd28c400016731170039ed1...,Аналитик данных,"Москва, м. Курская, готов к переезду (Москва),...",Опыт работы 6 лет 8 месяцев,"Мои хобби: инвестиции, автомобили, математика,...",Высшее образование\n2016\nМосковский государст...,"Специализации:\nАналитик\nBI-аналитик, аналити...","Работа с компьютерными программами, программы ...",Московский государственный технический универс...,NaN,...,-0.002009,0.002615,0.002877,-0.001430,0.003984,-0.002134,0.003607,0.001726,0.003098,0


In [9]:
df.index

RangeIndex(start=0, stop=2725, step=1)

**Перевод опыта работы в числовое значение:**

В данной функции мы приводим к единому виду опыт работы. Были найдены различные значения: years, year, months, лет, года и т. д. Мы учтем все найденные значения в коде.

In [5]:
#df.drop(list(df)[0], axis=1, inplace=True)
#df.drop(list(df)[0], axis=1, inplace=True)
#df.reset_index(inplace=True)
#df.drop(list(df)[0], axis=1, inplace=True)

In [22]:
df['Опыт'][2]
opyt_normalized = []
for x in df['Опыт']:
    x=x.removeprefix('Work experience ').removeprefix('Опыт работы')
    x=x.replace('years', ';')
    x=x.replace('year', ';')
    x=x.replace('года', ';')  # ВСЕГДА д. б. раньше
    x=x.replace('год', ';')
    x=x.replace('лет', ';')
    
    if x.find(';') == -1:
        x = '0;' + x
    x=x.removesuffix('months')
    x=x.removesuffix('month')
    x=x.removesuffix('месяцев')
    x=x.removesuffix('месяц')
    x=x.removesuffix('месяца')
     
    x=x.replace(' ', '')
    
    if re.search('[а–Я]+[a–Z]+', x):
        x='---'
    opyt_normalized.append(x)
    
df['Опыт нормализованный'] = opyt_normalized

In [23]:
df.loc[:, 'Опыт нормализованный'].sample(2)

777     12;5
1809    37;8
Name: Опыт нормализованный, dtype: object

**Сбор недостающих признаков:**

После сбора данных в ходе работы были придуманы еще некоторые признаки. Чтобы не запускать весь скрейпер заново, решено было добавить возможность их сбора не только в `scraping.ipynb`, но и в данный ноутбук. Код ниже собирает все недостающие признаки ("Где работал?", "Карьерный рост").

Соберем признак "Где работал?". Он реализован в `scraping.ipynb`, но был добавлен позже, чем были собраны данные, с которыми мы работаем.

In [21]:
driver = webdriver.Chrome('./chromedriver')
repet = []
for i in range(df.shape[0]):
    driver.get(df.iloc[i]['Ссылка'])
    work = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]')
    w = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]/div[@class="bloko-text bloko-text_strong"][not(@data-qa="resume-block-experience-position")]//span')
    
    work_list = []
    for ww in w:
        try:
            work_list.append(ww.text)
        except:
            1==1
    work_count = len(work)
    repet.append(work_list)

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_90561/3157379002.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [26]:
df['Где работал?'] = repet

Добавим еще один признак — карьерный рост: есть ли повторения в опыте работы? (Тоже наверх это)

In [45]:
driver = webdriver.Chrome('./chromedriver')
repet = []

for i in range(df.shape[0]):
    driver.get(df.iloc[i]['Ссылка'])
    work = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]')
    w = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]/div[@class="bloko-text bloko-text_strong"][not(@data-qa="resume-block-experience-position")]//span')
    
    works = []
    for ww in w:
        try:
            works.append(ww.text)
        except:
            1==1
    
    works_normilized = []
    for w in works:
        works_normilized.append(morph.normal_forms(w)[0])
    
    repet.append(len(set(works_normilized)) - len(works_normilized) < 0)

проверить, правда ли так много людей с повышениями:

In [46]:
repet[:3]

[False, False, False]

теперь для нормализованных названий найти повторения: `len(set(works)) - len(works) > 0` — и чекнуть адекватность такого определения

In [47]:
df['Карьерный рост'] = repet

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/1520391026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Карьерный рост'] = repet


**Словарь образования:**

В данной функции мы составляем словарь образования для создания бинарного признака "ВУЗ входит/не входит в топ". Возможна небольшая погрешность (сложно учесть все возможные варианты написания вузов), однако эта погрешность будет скомпенсирована большим количеством других, более простых признаков. Поэтому на результат это не повлияет.

In [13]:
edu_list = pd.concat([df['Образование-1'], df['Образование-2']], axis=0).drop_duplicates().reset_index().drop('index', axis=1)
edu_list = list(edu_list[list(edu_list)[0]])
len(edu_list)

ys=[]

for x in edu_list:
    s=str(x)
    l = s.find(',')+1
    y=s[l:].lstrip(' ')
    print(end='')
    ys.append(y)
    
for i in range(len(ys)):
    y = ys[i]
    is_space = re.fullmatch('[А-я]+[ ][А-я ()-]+', y)
    if is_space:
        #print(y)
        1==1
    is_m = re.fullmatch('Mosc.*', y) or re.fullmatch('Моск.*', y) or re.fullmatch('Петерб.*', y) or re.fullmatch('Peters.*', y) or re.fullmatch('СПб.*', y) or re.fullmatch('Новосиб.*', y) or re.fullmatch('Novosib.*', y) or re.fullmatch('Томск.*', y) or re.fullmatch('Tomsk.*', y)
    if not is_space and not is_m:
        ys[i] = 'NO'
#ys

cnt=0
for y in ys:
    if y != 'NO':
        cnt+=1
cnt

import numpy as np
ys = np.array(ys)
ys=np.unique(ys)
ys=list(ys)
len(ys)

"""
Топ-10 IT-вузов России по версии QS (Quacquarelli Symonds)

1. Московский государственный университет им. М. В. Ломоносова;
2. Университет ИТМО;
3. Московский физико-технологический институт (МФТИ);
4. Высшая школа экономики;
5. Санкт-Петербургский политехнический университет Петра Великого; 
6. Московский государственный технический университета (МГТУ) имени Н.Э. Баумана;
7. Санкт-Петербургский политехнический университет Петра Великого; 
8. Новосибирский государственный университет;
9. Московский инженерно-физический институт (МИФИ);
10. Томский политехнический университет.

https://propostuplenie.ru/article/spisok-luchshih-informacionnyh-vuzov-rossii/
"""

def is_top(y):
    if re.search('ВШЭ', y) or re.search('Высшая школа экономики', y) or re.search('HSE', y):
        y='ВШЭ'
    elif re.search('Московский Государственный Университет$', y) or re.search('Московский Государственный Университет им[.  ени] (М.|Л)', y) or re.search('МГУ[^ А-я]|\(МГУ\)', y) or re.search('Moscow state University', y) or re.search('MSU', y) or re.search('МГУ', y):
        y='МГУ'
    elif re.search('ИТМО', y):
        y='ИТМО'
    elif re.search('МФТИ', y) or re.search('Московский физико-технологический институт', y):
        y='Физтех'
        # ЗФТШ при МФТИ
        # - Физтех
    elif re.search('СПБ Политех', y) or re.search('Санкт-Петербургский политехнический университет', y):
        y='Политех СПб'
    elif re.search('Бауман', y):
        y='Бауманка'
    elif re.search('НГУ', y) or re.search('Новосибирский государственный университет', y):
        y='НГУ'
        # НГУ Наталии Нестеровой
    elif re.search('МИФИ', y) or re.search('Московский инженерно-физический институт', y):
        y='МИФИ'
        # Какие-то Техникум ДИТИ НИЯУ МИФИ и прочее...
    elif re.search('ТомГУ', y) or re.search('Томский политехнический университет', y):
        y='ТомГУ'
    else:
        y='NO'
    return y != 'NO'
        
# Дальше завожу колонку: parsed_uni по этому правилу -- 1 если хотя бы 1 уник топовый
# И там заменяю значения и вывожу равно ли МГУ / етц.

xs = []
for x in df.index:
    stroke = df.iloc[x].loc[['Образование-1', 'Образование-2']]
    k2 = False
    if type(stroke[0]) != float:
        k1 = is_top(stroke[0])
    if type(stroke[1]) != float:
        k2 = is_top(stroke[1])
    xs.append(k1 or k2)
    
df['Топовость образования']=xs

Посмотрим на распределение признаков:

In [14]:
df['Топовость образования'].value_counts()

False    2430
True      295
Name: Топовость образования, dtype: int64

**Флаг, что текст на eng:**

Здесь мы определим, написан ли текст на английском языке. Сделаем это по разделам "Релокация?", "Коммандировка?", "Город?". Если эти признаки написаны на английском языке, то и все резюме будет на английском языке

In [15]:
ts = []

for i in range(df.shape[0]):
    if type(df['Город?'][i])!=float and type(df['Релокация?'][i])!=float and type(df['Коммандировка?'][i])!=float:  # not nan\
        t = re.search('[A-z]+', df['Город?'][i]) or re.search('[A-z]+', df['Релокация?'][i]) or re.search('[A-z]+', df['Коммандировка?'][i])
        if t:
            t = 'ENG'
        else:
            t = 'RU'

    # print(x, t)
    else:
        t = '---'
    ts.append(t)
        
df['is_eng?'] = ts

In [16]:
about = df['О себе'].loc[df['is_eng?'] == 'RU']

In [17]:
df = df.loc[df['is_eng?'] == 'RU']

Посмотрим, что получилось:

In [18]:
about

0       Уверенный пользователь ПК, Ms Word, Ms Excel, ...
1       Энергична, активна, в поиске новых возможносте...
2       опыт прямых продаж, ведение коммерческих перег...
3       Коммуникабелен\nУмение работать с возражениями...
4       Дисциплина - умение выполнять поставленные зад...
                              ...                        
2720    Уверенный пользователь ПК: MS Office (Word, Ex...
2721    Моя потребность в постоянном улучшении себя на...
2722                                                    .
2723                                                  ---
2724                                                  ---
Name: О себе, Length: 2725, dtype: object

Действительно, мы видим, что все было обработано верно.

**Справочник по интересам:**

Есть ли аналитические комбинации слов в интересах? То есть, есть ли слова "аналитик", "данные" и т. д. в разделе "Интересы". Используем простое правило и получаем бинарный признак.

In [19]:
inter_list = []

for inter in df['Интересы']:
    intr = inter.split('\n')
    if re.search('(Analyst)|(Аналитик данных)|(аналитик)|(Аналитик)', inter):
        inter_list.append('found smt by rule (Analyst)|(Аналитик данных)|(аналитик)|(Аналитик)')
    else:
        inter_list.append('---')
        
df['Справочник по интересам']= inter_list

**Извлечение сущностей "Коммандировка", "Навыки", "Кол-во работ", "Наличие комбинаций":**

На сайте hh.ru единое поле для трех объектов сразу. Разъединим их с учетом разных знаяений и того, что они перечислены просто через запятую. 

In [68]:
xs1 = []
xs2=[]
xs3=[]
for x in df['Коммандировка']:
    # english
    x1 = re.search('[A-z]+', x)
    x2 = re.search('[a-z ]*willing to relocate', x)
    x3 = re.search('[a-z ]*prepared for [a-z]* business trips', x)
    if x1:
        x1=x1[0]
    if x2:
        x2=x2[0]
    if x3:
        x3=x3[0]
    
    if x1 or x2 or x3:  # if found smt in engish — is it not wrong way to do it?
        xs1.append(x1)
        xs2.append(x2)
        xs3.append(x3)
    else:
        # russian — work in progress
        # Москва, не готова к переезду, готова к редким командировкам
        # Санкт-Петербург, м. Гражданский проспект, готов к переезду (Москва), готов к командировкам
        # Москва, готов к переезду (Нидерланды, Испания, Франция, Италия, Австрия, Германия, Швейцария), готов к командировкам
        # Москва, хочу переехать, готов к командировкам
        # Москва, м. Ясенево, не готов к переезду, готов к редким командировкам
        x1 = re.search('[А-я]+', x)  # is it correct?
        x2 = re.search('[А-я]*[ не]*готов[а]* к переезду', x)
        x3 = re.search('[А-я]*[ не]*готов[а]* к[ а-я]* командировкам', x)  # черновое правило
        if x1:
            x1=x1[0]
        if x2:
            x2=x2[0]
        if x3:
            x3=x3[0]
        xs1.append(x1)
        xs2.append(x2)
        xs3.append(x3)

zs = []
for z in df['Навыки']:
    zzz = z.split('\n')
    zs.append(zzz)
    
df['Навыки списком'] = zs
df['Релокация?'] = xs3
df['Коммандировка?'] = xs2
df['Город?'] = xs1

"""
1. Коммандировка 
2. Навыки
3. Кол-во работ
4. Наличие аналитических комбинаций слов в работе — новый столбец
"""

import numpy as np
import re

#print(df['Коммандировка'][0])
#print(re.search('[A-z]+', df['Коммандировка'][0Z]))
#print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][8]))
#print(re.search('[a-z ]*prepared for business trips', df['Коммандировка'][8]))

#print(df['Коммандировка'][0])
#print(re.search('[A-z]+', df['Коммандировка'][0]))
#print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][0]))
#print(re.search('[a-z ]*prepared for [a-z]* business trips', df['Коммандировка'][0]))

df['Навыки'].iloc[0].split('\n')  # вот так?

work_list = df['Где работал?']
analytics_words = []
works_cnt = []
skills_list = df['Навыки']

for work in skills_list:
    # пока супер примитивное правило:
    if re.search('(SQL)|(Excel)|(Python)|(Tableau)|(Power BI)', str(work)):
        analytics_words.append('found smt by rule (SQL)|(Excel)|(Python)|(Tableau)|(Power BI)')
    else:
        analytics_words.append('---')

for work in work_list:
    works_cnt.append(len(work))
df['Кол-во работ'] = works_cnt
    
df['Аналитические комбинации слов в навыках'] = analytics_words

Посмотрим, к примеру, сколько было найдено аналитических комбинаций:

In [70]:
df['Аналитические комбинации слов в навыках'].value_counts()

---                                                              2241
found smt by rule (SQL)|(Excel)|(Python)|(Tableau)|(Power BI)     484
Name: Аналитические комбинации слов в навыках, dtype: int64

**Перевод уровня образования:**

Определим уровень образования, исходя из выделенных возможных значений: Бакалавр, bachelor, Secondary education и т. д.

In [32]:
z = list(np.unique(list(df['Уровень образование'])))

for y in z:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    print(y)
    
ys=[]
for y in df['Уровень образование']:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    ys.append(y)
df['Уровень оборазования нормализованный'] = ys

Высшее образование
Бакалавр
Доктор наук
Кандидат наук
Магистр
Неоконченное высшее образование
Образование
Среднее образование
Среднее специальное образование


**Сохранение результатов:**

Сохраним полученные результаты:

In [65]:
df.to_csv('resumes_features.csv')

**mBART перед построением токенов в "О себе"**

Перед токенизацией суммаризуем текст, чтобы выделить основную мысль и убрать "шум" — так будет проще работать с ним. Для этого воспользуемся алгоритмом [mBART](https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta?text=%D0%97%D0%B0+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F+%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B+%D0%B8+%D1%83%D1%87%D1%91%D0%B1%D1%8B%2C+%D1%8F+%D0%BE%D0%B2%D0%BB%D0%B0%D0%B4%D0%B5%D0%BB+%D0%BA%D0%BE%D0%BC%D0%BF%D0%B5%D1%82%D0%B5%D0%BD%D1%86%D0%B8%D1%8F%D0%BC%D0%B8%2C+%D0%BD%D0%B5%D0%BE%D0%B1%D1%85%D0%BE%D0%B4%D0%B8%D0%BC%D1%8B%D0%BC%D0%B8+%D0%B4%D0%BB%D1%8F+%D0%B8%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%2C+%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F+%D0%B8+%D1%83%D1%81%D0%BF%D0%B5%D1%88%D0%BD%D0%BE%D0%B3%D0%BE+%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F+%D1%86%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D1%8B%D1%85+%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B9+%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%2C+%D1%83%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F%2C+%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0+%D0%B8+%D1%81%D0%B8%D0%BD%D1%82%D0%B5%D0%B7%D0%B0+%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D1%85+%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B9+%D0%B2+%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B5%D1%81%D0%B0%D1%85+%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F+%D1%81%D0%BB%D0%BE%D0%B6%D0%BD%D0%BE%D0%B9+%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%86%D0%B8%D0%B8.%0A%D0%AF+%D0%BE%D0%B1%D0%BB%D0%B0%D0%B4%D0%B0%D1%8E+%D1%88%D0%B8%D1%80%D0%BE%D0%BA%D0%B8%D0%BC+%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D0%BE%D0%BC+%D0%B2+%D0%BF%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B8+BI+%D0%BE%D1%82%D1%87%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8%2C+%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8%2C+%D1%81%D0%B2%D1%8F%D0%B7%D1%8F%D1%85%2C+%D0%BA%D0%BE%D1%80%D1%80%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85+%D0%B2+Power+Qwery+%D0%B8+%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B+%D1%81+%D0%B1%D0%B0%D0%B7%D0%B0%D0%BC%D0%B8+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.%0A%D0%A3%D1%87%D0%B0%D1%81%D1%82%D0%B2%D0%BE%D0%B2%D0%B0%D0%BB+%D0%B2+%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B0%D1%85+%D0%BF%D0%BE+%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8+%D0%B8+%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8+%D0%B1%D0%B8%D0%B7%D0%BD%D0%B5%D1%81-%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D0%BE%D0%B2+%D0%B8+%D0%BE%D1%82%D1%87%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8.%0A%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%BB+%D0%B2+%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D1%8F%D1%85+%D1%81+%D1%82%D0%B0%D0%BA%D0%B8%D0%BC%D0%B8+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D0%BC%D0%B8+%D0%BA%D0%B0%D0%BA+%3A+Power+BI%2C+Excel%2C+MS+Office%2C+VipPlanner.%0A%D0%97%D0%BD%D0%B0%D0%BA%D0%BE%D0%BC+%D1%81+%D1%82%D0%B0%D0%BA%D0%B8%D0%BC%D0%B8+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%BD%D1%8B%D0%BC%D0%B8+%D0%B8%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D0%BC%D0%B8+%D0%BA%D0%B0%D0%BA+%3A+Jira%2C+kanban%2C+confluence.%0A%D0%9F%D1%80%D0%B8%D0%BC%D0%B5%D1%80%D1%8B+%D0%B4%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4%D0%BE%D0%B2+%D0%BC%D0%BE%D0%B3%D1%83+%D0%B2%D1%8B%D1%81%D0%BB%D0%B0%D1%82%D1%8C+%D0%BB%D0%B8%D1%87%D0%BD%D0%BE.%0A%D0%91%D1%83%D0%B4%D1%83+%D1%80%D0%B0%D0%B4+%D0%B2+%D1%80%D0%B0%D0%BC%D0%BA%D0%B0%D1%85+%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F+%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%B8%D1%82%D1%8C+%D0%BD%D0%B0+%D0%B2%D1%81%D0%B5+%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B5%D1%81%D1%83%D1%8E%D1%89%D0%B8%D0%B5+%D0%92%D0%B0%D1%81+%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D1%8B.%0A%D0%97%D0%B0%D1%80%D0%B0%D0%BD%D0%B5%D0%B5+%D0%B1%D0%BB%D0%B0%D0%B3%D0%BE%D0%B4%D0%B0%D1%80%D1%8E+%D0%92%D0%B0%D1%81+%D0%B7%D0%B0+%D1%83%D0%B4%D0%B5%D0%BB%D1%91%D0%BD%D0%BD%D0%BE%D0%B5+%D0%BC%D0%BE%D0%B5%D0%B9+%D0%BA%D0%B0%D0%BD%D0%B4%D0%B8%D0%B4%D0%B0%D1%82%D1%83%D1%80%D0%B5+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F.%0A%D0%92%D0%BB%D0%B0%D0%B4%D0%B5%D1%8E+%D0%BD%D0%B0%D0%B2%D1%8B%D0%BA%D0%BE%D0%BC+%22%D0%94%D0%B5%D0%BB%D0%BE%D0%B2%D0%B0%D1%8F+%D0%BF%D0%B5%D1%80%D0%B5%), предобученном на датасете Gazeta.

Импорт модели:

In [53]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [54]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

Функция для суммаризации текста с использованием модели:

In [55]:
def summarize(article_text):
    input_ids = tokenizer(
        [article_text],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

Запускаем функцию:

In [62]:
summarizations = []
for i in range(df.shape[0]):
    if df['О себе'].iloc[i] != '---' and df['is_eng?'].iloc[i] == 'RU' and (not pd.isna(df['О себе'].iloc[i])):
        text_to_summarize = df['О себе'].iloc[i]
        summarizations.append(summarize(text_to_summarize))
    else:
        summarizations.append(df['О себе'].iloc[i])
    
    if len(summarizations) % 5 == 0:
        print(len(summarizations), '/', df.shape[0])

5 / 2725
10 / 2725
15 / 2725
20 / 2725
25 / 2725
30 / 2725
35 / 2725
40 / 2725
45 / 2725
50 / 2725
55 / 2725
60 / 2725
65 / 2725
70 / 2725
75 / 2725
80 / 2725
85 / 2725
90 / 2725
95 / 2725
100 / 2725
105 / 2725
110 / 2725
115 / 2725
120 / 2725
125 / 2725
130 / 2725
135 / 2725
140 / 2725
145 / 2725
150 / 2725
155 / 2725
160 / 2725
165 / 2725
170 / 2725
175 / 2725
180 / 2725
185 / 2725
190 / 2725
195 / 2725
200 / 2725
205 / 2725
210 / 2725
215 / 2725
220 / 2725
225 / 2725
230 / 2725
235 / 2725
240 / 2725
245 / 2725
250 / 2725
255 / 2725
260 / 2725
265 / 2725
270 / 2725
275 / 2725
280 / 2725
285 / 2725
290 / 2725
295 / 2725
300 / 2725
305 / 2725
310 / 2725
315 / 2725
320 / 2725
325 / 2725
330 / 2725
335 / 2725
340 / 2725
345 / 2725
350 / 2725
355 / 2725
360 / 2725
365 / 2725
370 / 2725
375 / 2725
380 / 2725
385 / 2725
390 / 2725
395 / 2725
400 / 2725
405 / 2725
410 / 2725
415 / 2725
420 / 2725
425 / 2725
430 / 2725
435 / 2725
440 / 2725
445 / 2725
450 / 2725
455 / 2725
460 / 2725
465 / 27

Сохраняем результат:

In [63]:
df['О себе, summarized'] = summarizations

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37832/1096740388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['О себе, summarized'] = summarizations


In [71]:
df.to_csv('resumes_features.csv')

**Разметка данных:**

Буду выбирать случайное резюме и оценивать. Всего планируется оценить ~4% резюме, то есть, ~100 штук. 

In [ ]:
df = pd.read_csv('resumes_features.csv')

In [32]:
df['DA'] = np.nan
df['DS'] = np.nan

In [33]:
right_count = 0
false_count = 0
print('reset true, false')

reset true, false


**!!** Смотрим на случайное резюме:

In [1681]:
df_sample = df.sample(1).loc[:, ['Ссылка', 'Название', 'DA', 'DS']]
print(df_sample['Ссылка'].iloc[0])
print(df_sample['Название'].iloc[0])

https://hh.ru/resume/54f904d500081af3870039ed1f6b6479456253?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA+%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B6&source=search&hhtmFrom=resumes_catalog
Менеджер по работе с маркетплейсами


Размечаем:

In [1682]:
df_sample['DA'] = 0

In [1683]:
df_sample['DS'] = 0

In [1684]:
df.loc[df_sample.index[0], 'DA'] = 0
df.loc[df_sample.index[0], 'DS'] = 0

Проверка:

In [1685]:
df_sample.iloc[0].loc[['Название', 'DA', 'DS']]

Название    Менеджер по работе с маркетплейсами
DA                                            0
DS                                            0
Name: 1951, dtype: object

In [1686]:
df.loc[df_sample.index[0], ['Название', 'DA', 'DS']]

Название    Менеджер по работе с маркетплейсами
DA                                          0.0
DS                                          0.0
Name: 1951, dtype: object

In [1687]:
print('false:', false_count, '->', end=' ')
false_count += 1
print(false_count)

false: 67 -> 68


Отмечаем (False ~ not DA and not BA):

In [1663]:
print('true:', right_count, '->', end=' ')
right_count += 1
print(right_count)

true: 32 -> 33


In [1688]:
print(right_count + false_count)

101


In [24]:
df.to_csv('resumes_features.csv')

Проверим, сколько мы разметили в итоге:

In [1692]:
df.loc[:, ['DA', 'DS']].value_counts().sum()

100